## Toronto Neighborhood Data-analysis Retrieve Data from Wikipedia page

### Install wikipedia if not install

In [ ]:
#pip install wikipedia

### Import Library and read page from Wikipedia

In [5]:
import pandas as pd
import wikipedia as wp
 
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df_input = pd.read_html(html)[0]
df_input.to_csv('Toronto_Data.csv',header=0,index=False)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
df_input.columns=['Postcode','Borough','Neighbourhood']
df_input.drop([0],axis=0,inplace=True)

### Ignore cells with a borough that is Not assigned 

In [7]:
df_toronto = pd.DataFrame(df_input[df_input["Borough"] != "Not assigned"])

### Rename Postcode as PostalCode

In [8]:
df_toronto.rename(columns={"Postcode": "PostalCode"}, inplace = True)

### Group more than one neighborhood can exist in one postal code area.

In [9]:
df_toronto_group=df_toronto.groupby("PostalCode").agg(lambda x:','.join(set(x)))
df_toronto_group.reset_index(drop=False, inplace=True)

### Neighborhood will be the same as the borough if not assigned.

In [10]:
df_toronto_group.loc[df_toronto_group['Neighbourhood']=="Not assigned",'Neighbourhood']=df_toronto_group.loc[df_toronto_group['Neighbourhood']=="Not assigned",'Borough']
df_toronto_group.head(12)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park"
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


### The number of rows in DataFrame

In [11]:
df_toronto_group.shape

(103, 3)

## Using the link to a csv file that has the geographical coordinates of each postal code

In [4]:
df_Geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
df_Geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_toronto_merge = df_toronto_group.merge(df_Geospatial_data, left_on='PostalCode', right_on='Postal Code')
df_toronto_merge.drop(columns=['Postal Code'], inplace = True)
df_toronto_merge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
